In [32]:
from serial import Serial
from pyubx2 import UBXReader, NMEA_PROTOCOL, UBX_PROTOCOL
from pyubx2 import UBXMessage, SET_LAYER_RAM, TXN_NONE, POLL, GET
from pyubx2 import POLL_LAYER_RAM

In [2]:
usb_port = '/dev/ttyACM0'
baud_rate = 38400
set_layers = SET_LAYER_RAM
poll_layers = POLL_LAYER_RAM
transaction = TXN_NONE

In [3]:
def msg_send(msg):
    with Serial(usb_port, baud_rate, timeout=3) as stream:
        stream.write(msg)
        
def ubx_set(cfgData, layers=set_layers, transaction=transaction, verbose=False):
    msg = UBXMessage.config_set(layers, transaction, cfgData)
    if verbose:
        print(msg)
    else:
        msg_send(msg.serialize())

def ubx_poll(keys, layers=poll_layers,position=0, verbose=False):
    msg = UBXMessage.config_poll(layers, position, keys)
    if verbose:
        print(msg)
    else:
        with Serial(usb_port, baud_rate, timeout=3) as stream:
            stream.write(msg.serialize())
            ubr = UBXReader(stream, protfilter=NMEA_PROTOCOL | UBX_PROTOCOL)
            raw_data, parsed_data = ubr.read()
            return parsed_data

def ubx_reader(id=None, usb_port=usb_port, baud_rate=baud_rate, verbose=False, timeout=10):
    with Serial(usb_port, baud_rate, timeout=3) as stream:
        t = 0
        while True:
            t = t + 1
            if t > timeout:
                return None
            ubr = UBXReader(stream, protfilter=NMEA_PROTOCOL | UBX_PROTOCOL)
            raw_data, parsed_data = ubr.read()
            if parsed_data is not None:
                if (id is None or parsed_data.identity == id) and verbose:
                    print(parsed_data)
                if id is None or parsed_data.identity == id:
                    return parsed_data
                else:
                    continue


In [43]:
FixTypeList = ['no-fix', 'dead-reckoning-only', '2D-fix', '3D-fix', 'GPS + dead-reckoning', 'Time-only-fix']
SvinVldStatusList = ['not valid', 'valid']
SvinActiveStatusList = ['stop', 'in-progress']

def poll_nav_clk():
    msg = UBXMessage(b'\x01', b'\x22', POLL)
    msg_send(msg.serialize())
    msg = ubx_reader(id='NAV-CLOCK', verbose=True)
    return msg

def get_svin_status():
    msg = ubx_reader(id='TIM-SVIN')
    return SvinVldStatusList[msg.valid], SvinActiveStatusList[msg.active]

def get_fixtype(): 
    msg = ubx_reader(id='NAV-PVT')
    return FixTypeList[msg.fixType]

def set_coordinate(lon, lat, alt):
    cfgData = [('CFG_TMODE_LON', int(lon*10**7)), ('CFG_TMODE_LON_HP', int(lon*10**9)), 
            ('CFG_TMODE_LAT', int(lat*10**7)), ('CFG_TMODE_LAT_HP', int(lat*10**9)),
            ('CFG-TMODE-HEIGHT', int(alt*100)), ('CFG-TMODE-HEIGHT_HP', int(alt*10**4))]
    ubx_set(cfgData)
    
def set_acc(acc):
    cfgData = ['CFG-TMODE-FIXED_POS_ACC', acc*10**4]
    ubx_set(cfgData)

def set_rtcm():
    cfgData = [('CFG_MSGOUT_RTCM_3X_TYPE1005_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1077_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1087_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1097_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1127_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1230_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE4072_1_USB', 1)]
    ubx_set(cfgData)

def get_coordindate():
    msg = ubx_reader(id='NAV-PVT')
    lon = msg.lon
    lat = msg.lat
    alt = msg.height
    return lon, lat, alt

def get_acc():
    msg = ubx_reader(id='NAV-PVT')
    hacc = msg.hAcc/1000
    vacc = msg.vAcc/1000
    return hacc, vacc

def set_gnss():
    cfgData = [ ('CFG_SIGNAL_GPS_ENA', 1), 
                ('CFG_SIGNAL_SBAS_ENA', 0),
                ('CFG_SIGNAL_GAL_ENA', 1),
                ('CFG_SIGNAL_BDS_ENA', 1), 
                ('CFG_SIGNAL_QZSS_ENA', 0), 
                ('CFG_SIGNAL_GLO_ENA', 0),
                ('CFG_SIGNAL_NAVIC_ENA', 0)]
    ubx_set(cfgData)
    
def master_io_msg():
    cfgData = [('CFG_USBINPROT_UBX',  1),
               ('CFG_USBINPROT_NMEA', 1),
               ('CFG_USBOUTPROT_UBX', 0),
               ('CFG_USBOUTPROT_NMEA',0),
               ('CFG_USBOUTPROT_RTCM',1)]
    ubx_set(cfgData)

def slave_io_msg():
    cfgData = [('CFG_USBINPROT_UBX',  1),
               ('CFG_USBINPROT_NMEA', 1),
               ('CFG_USBINPROT_RTCM', 1),
               ('CFG_USBOUTPROT_UBX', 1),
               ('CFG_USBOUTPROT_NMEA',1)]
    ubx_set(cfgData)

In [5]:
# USB input: NMEA, UBX 
keys = [("CFG_USBINPROT_NMEA", 1), ("CFG_USBINPROT_UBX", 1)]
ubx_set(keys, verbose=True)
# USB output: UBX
keys = [("CFG_USBOUTPROT_NMEA", 0), ("CFG_USBOUTPROT_UBX", 1)]
ubx_set(keys, verbose=False)



<UBX(CFG-VALSET, version=0, ram=1, bbr=0, flash=0, action=0, reserved0=0, CFG_USBINPROT_NMEA=1, CFG_USBINPROT_UBX=1)>


In [6]:
keys=["CFG_USBOUTPROT_NMEA"]
ubx_poll(keys, verbose=True)

<UBX(CFG-VALGET, version=0, layer=0, position=0, keys_01=276299778)>


In [7]:
keys=["CFG_USBOUTPROT_NMEA"]
ubx_poll(keys)

UBXMessage(b'\x06', b'\x8b', 0, payload=b'\x01\x00\x00\x00\x02\x00x\x10\x00')

In [8]:
# set the GPS receiver work in survey-in mode
# survey-in minimum duration is 1200s
# survey-in position accuracy limit 0.1m
keys = [("CFG_TMODE_MODE", 1), ("CFG_TMODE_SVIN_MIN_DUR", 1200), ("CFG_TMODE_SVIN_ACC_LIMIT", 1000)]
ubx_set(keys, verbose=False)

In [9]:
# enable ubx-pvt message out from usb
keys = [("CFG_MSGOUT_UBX_NAV_PVT_USB",1), ("CFG_MSGOUT_UBX_TIM_SVIN_USB", 1)]
ubx_set(keys)

In [10]:
msg = ubx_reader(verbose=True)
#print(msg.gnssFixOk)

<UBX(NAV-PVT, iTOW=19:42:13, year=2024, month=6, day=26, hour=19, min=42, second=13, validDate=1, validTime=1, fullyResolved=1, validMag=0, tAcc=22, nano=-254258, fixType=3, gnssFixOk=1, diffSoln=0, psmState=0, headVehValid=0, carrSoln=0, confirmedAvai=1, confirmedDate=1, confirmedTime=1, numSV=25, lon=-122.2570362, lat=37.8728622, height=90641, hMSL=120074, hAcc=526, vAcc=927, velN=-5, velE=0, velD=17, gSpeed=5, headMot=0.0, sAcc=12, headAcc=167.121, pDOP=1.05, invalidLlh=0, lastCorrectionAge=0, reserved0=659535132, headVeh=0.0, magDec=0.0, magAcc=0.0)>


In [11]:
msg = ubx_reader(verbose=True)

<UBX(NAV-PVT, iTOW=19:42:17, year=2024, month=6, day=26, hour=19, min=42, second=17, validDate=1, validTime=1, fullyResolved=1, validMag=0, tAcc=22, nano=-254492, fixType=3, gnssFixOk=1, diffSoln=0, psmState=0, headVehValid=0, carrSoln=0, confirmedAvai=1, confirmedDate=1, confirmedTime=1, numSV=25, lon=-122.2570362, lat=37.872862, height=90572, hMSL=120005, hAcc=519, vAcc=917, velN=-5, velE=0, velD=17, gSpeed=5, headMot=0.0, sAcc=11, headAcc=167.121, pDOP=1.05, invalidLlh=0, lastCorrectionAge=0, reserved0=659535132, headVeh=0.0, magDec=0.0, magAcc=0.0)>


In [12]:
ubx_reader(id='TIM-SVIN' ,verbose=True)

<UBX(TIM-SVIN, dur=12, meanX=-269056854, meanY=-426313288, meanZ=389436981, meanV=1106060, obs=13, valid=0, active=1, reserved1=0)>


UBXMessage(b'\r', b'\x04', 0, payload=b'\x0c\x00\x00\x00\xaa\x84\xf6\xef\xb8\xf9\x96\xe65V6\x17\x8c\xe0\x10\x00\r\x00\x00\x00\x00\x01\x00\x00')

In [13]:
# enable ubx messages
keys = [("CFG_MSGOUT_UBX_NAV_PVT_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_SAT_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_SIG_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_STATUS_USB", 1),
        ("CFG_MSGOUT_UBX_RXM_RTCM_USB", 1)]
ubx_set(keys)

In [14]:
ubx_reader(id='NAV-PVT' ,verbose=True)

<UBX(NAV-PVT, iTOW=19:42:28, year=2024, month=6, day=26, hour=19, min=42, second=28, validDate=1, validTime=1, fullyResolved=1, validMag=0, tAcc=22, nano=-255133, fixType=3, gnssFixOk=1, diffSoln=0, psmState=0, headVehValid=0, carrSoln=0, confirmedAvai=1, confirmedDate=1, confirmedTime=1, numSV=25, lon=-122.257036, lat=37.8728616, height=90419, hMSL=119852, hAcc=514, vAcc=897, velN=-2, velE=1, velD=14, gSpeed=2, headMot=0.0, sAcc=29, headAcc=167.121, pDOP=1.05, invalidLlh=0, lastCorrectionAge=0, reserved0=659535132, headVeh=0.0, magDec=0.0, magAcc=0.0)>


UBXMessage(b'\x01', b'\x07', 0, payload=b"\xf0\xee\xad\x13\xe8\x07\x06\x1a\x13*\x1c7\x16\x00\x00\x00c\x1b\xfc\xff\x03\x01\xea\x19\x88\x0e!\xb7\xa8\xf0\x92\x163a\x01\x00,\xd4\x01\x00\x02\x02\x00\x00\x81\x03\x00\x00\xfe\xff\xff\xff\x01\x00\x00\x00\x0e\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x1d\x00\x00\x00\xa4\x01\xff\x00i\x00\x00\x00\x1c\xb5O'\x00\x00\x00\x00\x00\x00\x00\x00")

In [16]:
get_fixtype()

'3D-fix'

In [29]:
get_svin_status()

('not valid', 'in-progress')

In [26]:
get_coordindate()

(-122.2570476, 37.8728563, 91.67)

In [28]:
get_acc()

(0.515, 0.899)

In [20]:
poll_nav_clk()

<UBX(NAV-CLOCK, iTOW=19:42:54, clkB=256643, clkD=58, tAcc=2, fAcc=114)>


UBXMessage(b'\x01', b'"', 0, payload=b'\x80T\xae\x13\x83\xea\x03\x00:\x00\x00\x00\x02\x00\x00\x00r\x00\x00\x00')

In [45]:
keys = ['CFG_SIGNAL_GPS_ENA', 
        'CFG_SIGNAL_SBAS_ENA',
        'CFG_SIGNAL_GAL_ENA',
        'CFG_SIGNAL_BDS_ENA', 
        'CFG_SIGNAL_QZSS_ENA', 
        'CFG_SIGNAL_GLO_ENA',
        'CFG_SIGNAL_NAVIC_ENA']
msg = ubx_poll(keys)
print(msg)

<UBX(CFG-VALGET, version=1, layer=0, position=0, CFG_SIGNAL_GPS_ENA=1, CFG_SIGNAL_SBAS_ENA=0, CFG_SIGNAL_GAL_ENA=1, CFG_SIGNAL_BDS_ENA=1, CFG_SIGNAL_QZSS_ENA=0, CFG_SIGNAL_GLO_ENA=0, CFG_SIGNAL_NAVIC_ENA=0)>


In [44]:
set_gnss()